In [35]:
import pandas as pd
import numpy as np

In [36]:
df=pd.read_csv("/home/sebastian/Downloads/Temperaturas.zip",index_col=0)
df.head()

,sensor,fecha,temperatura
id,,,
2,sala,2020-03-31 14:38:14,23.80
3,oficina,2020-03-31 14:43:40,27.20
4,piscina,2020-03-31 14:46:26,27.62
5,sala,2020-03-31 14:48:14,25.40
6,oficina,2020-03-31 14:53:40,28.80


In [37]:
ndf=df[df["sensor"]=='piscina']
ndf.head()

,sensor,fecha,temperatura
id,,,
4,piscina,2020-03-31 14:46:26,27.62
7,piscina,2020-03-31 14:56:32,27.56
10,piscina,2020-03-31 15:06:37,27.62
13,piscina,2020-03-31 15:16:42,27.62
16,piscina,2020-03-31 15:26:47,27.62


In [38]:
ndf.dtypes

sensor          object
fecha           object
temperatura    float64
dtype: object

In [39]:
ndf.head()

,sensor,fecha,temperatura
id,,,
4,piscina,2020-03-31 14:46:26,27.62
7,piscina,2020-03-31 14:56:32,27.56
10,piscina,2020-03-31 15:06:37,27.62
13,piscina,2020-03-31 15:16:42,27.62
16,piscina,2020-03-31 15:26:47,27.62


In [40]:
ndf.dtypes

sensor          object
fecha           object
temperatura    float64
dtype: object

In [41]:

ndf.drop(['sensor'], axis=1, inplace=True)
#ndf

/tmp/ipykernel_29697/3796156364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf.drop(['sensor'], axis=1, inplace=True)


In [42]:
datetime_index = pd.DatetimeIndex(ndf.fecha.values)

df2=ndf.set_index(datetime_index)

In [43]:
df2.drop('fecha', axis=1, inplace=True)


In [44]:
df2.head()

,temperatura
2020-03-31 14:46:26,27.62
2020-03-31 14:56:32,27.56
2020-03-31 15:06:37,27.62
2020-03-31 15:16:42,27.62
2020-03-31 15:26:47,27.62


In [45]:
#df2.set_index('fecha', inplace=True)
# Taking mean values for a frequency of 2 minutes
df_group = df2.groupby(pd.Grouper( freq='10T'))['temperatura'].agg('mean')   
df_group.dropna(inplace=True)
df_group = df_group.to_frame().reset_index()
#print(df_group)
#ndf.info()



In [47]:
df_group.head()

,index,temperatura
0,2020-03-31 14:40:00,27.62
1,2020-03-31 14:50:00,27.56
2,2020-03-31 15:00:00,27.62
3,2020-03-31 15:10:00,27.62
4,2020-03-31 15:20:00,27.62
